In [2]:
from spark_tools import *

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import matplotlib as mpl
#import seaborn as sns

import apollo
from apollo import OverrideConfiguration
from apollo import dataset
import apollo_artifacts
from apollo_artifacts import datasets


In [3]:
import pygsheets

def gsheets_client() -> pygsheets.client.Client:
    client_secret = '{"installed":{"client_id":"721835620182-1bnpnhso6blqgnud8p28eo635cvprd24.apps.googleusercontent.com","project_id":"circleup-dev","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://accounts.google.com/o/oauth2/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"4QVTGL6G2ktoF1_pQiUavVnQ","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}'
    open('/tmp/client_secret.json', 'w').write(client_secret)
    return pygsheets.authorize(outh_file='/tmp/client_secret.json', outh_nonlocal=True)
gsheets_client()

def write_google_sheet(url, *, df, title, mode='overwrite'):
    gs = gsheets_client()
    spreadsheet = gs.open_by_url(url)

    header = [str(x) for x in df.columns]
    values = df.rdd.map(lambda row: [str(x) if x else '' for x in row]).collect()

    if mode == 'overwrite':
        try:
            worksheet = spreadsheet.worksheet_by_title(title)
        except WorksheetNotFound:
            worksheet = None
    else:
        worksheet = None

    if worksheet is None:
        worksheet = spreadsheet.add_worksheet(title=title, rows=1, cols=len(header))

    worksheet.resize(rows=1, cols=len(header))
    worksheet.update_row(index=1, values=header)
    worksheet.insert_rows(row=1, values=values, inherit=True)
    return worksheet

In [4]:
# look in https://s3.console.aws.amazon.com/s3/buckets/circleup-datascience?prefix=HGP/ for filepaths with hgp dimensions outputs

In [5]:
paths = ['s3a://circleup-datascience/HGP/2021-03-04/combined_hgp_dimensions_scored_output_202101',
        's3a://circleup-datascience/HGP/2021-01-07/combined_hgp_dimensions_scored_output_202012',
        's3a://circleup-datascience/HGP/2020-12-16/combined_hgp_dimensions_scored_output_202011',
        's3a://circleup-datascience/HGP/2020-11-12/combined_hgp_dimensions_scored_output_202010',
        's3a://circleup-datascience/HGP/2020-10-15/combined_hgp_dimensions_scored_output_202009',
        's3a://circleup-datascience/HGP/2020-09-22/combined_hgp_dimensions_scored_output_202008',
        's3a://circleup-datascience/HGP/2020-09-08/combined_hgp_dimensions_scored_output_202007',
        's3a://circleup-datascience/HGP/2020-07-06/combined_hgp_dimensions_scored_output_202005',
        's3a://circleup-datascience/HGP/2020-06-03/combined_hgp_dimensions_scored_output_202004',
        's3a://circleup-datascience/HGP/2020-05-04/combined_hgp_dimensions_scored_output_202003',
        's3a://circleup-datascience/HGP/2020-04-09/combined_hgp_dimensions_scored_output_202002'] 

In [6]:
hgp_dict={}

In [8]:
for path in paths:
    print("Reading: " + str(path))
    hgp_dict[path]= spark.read.parquet(path)
    print("Values")
    print(hgp_dict)

Reading: s3a://circleup-datascience/HGP/2021-03-04/combined_hgp_dimensions_scored_output_202101
Values
{'s3a://circleup-datascience/HGP/2021-03-04/combined_hgp_dimensions_scored_output_202101': DataFrame[normalized_url: string, fct_date: string, score_date: string, parent_category: string, parent_category_hgp_score: double, overall_hgp_score: double, overall_feature_density: double, t12m_nielsen_sales: double, t12m_nielsen_sales_pct_growth: double, nielsen_feature_density: double, nielsen_score: double, parent_category_nielsen_score: double, nielsen_weight: double, offline_distribution_feature_density: double, offline_distribution_score: double, parent_category_offline_distribution_score: double, offline_distribution_weight: double, reviews_feature_density: double, reviews_score: double, parent_category_reviews_score: double, reviews_weight: double, social_feature_density: double, social_score: double, parent_category_social_score: double, social_weight: double, web_feature_density: do

Values
{'s3a://circleup-datascience/HGP/2021-03-04/combined_hgp_dimensions_scored_output_202101': DataFrame[normalized_url: string, fct_date: string, score_date: string, parent_category: string, parent_category_hgp_score: double, overall_hgp_score: double, overall_feature_density: double, t12m_nielsen_sales: double, t12m_nielsen_sales_pct_growth: double, nielsen_feature_density: double, nielsen_score: double, parent_category_nielsen_score: double, nielsen_weight: double, offline_distribution_feature_density: double, offline_distribution_score: double, parent_category_offline_distribution_score: double, offline_distribution_weight: double, reviews_feature_density: double, reviews_score: double, parent_category_reviews_score: double, reviews_weight: double, social_feature_density: double, social_score: double, parent_category_social_score: double, social_weight: double, web_feature_density: double, web_score: double, parent_category_web_score: double, web_weight: double, overall_confiden

KeyboardInterrupt: 

In [35]:
df_read = (cu_read_google_sheet(url='https://docs.google.com/spreadsheets/d/1RwpsFUyXEMCMLqVdfK5gNrpcNgcobhqfC8yR9hxcXsw/edit#gid=343650079',
                               sheet='cbd').select('url').withColumn('normalized_url', c.url))

In [36]:
df_read = ['']

root
 |-- url: string (nullable = true)
 |-- normalized_url: string (nullable = true)



In [7]:
filtered_df_dict={}

In [13]:
for key in hgp_dict.keys():
    print(key)
    filtered_df_dict[key]=(hgp_dict[key]
                           #.join(df_read, on = ['normalized_url'])
                           .where(F.col('normalized_url')=='foodstirs.com')
                           .select(['normalized_url', 'fct_date', 'parent_category_hgp_score']).toPandas())

s3a://circleup-datascience/HGP/2021-03-04/combined_hgp_dimensions_scored_output_202101
s3a://circleup-datascience/HGP/2021-01-07/combined_hgp_dimensions_scored_output_202012
s3a://circleup-datascience/HGP/2020-12-16/combined_hgp_dimensions_scored_output_202011
s3a://circleup-datascience/HGP/2020-11-12/combined_hgp_dimensions_scored_output_202010
s3a://circleup-datascience/HGP/2020-10-15/combined_hgp_dimensions_scored_output_202009
s3a://circleup-datascience/HGP/2020-09-22/combined_hgp_dimensions_scored_output_202008
s3a://circleup-datascience/HGP/2020-09-08/combined_hgp_dimensions_scored_output_202007
s3a://circleup-datascience/HGP/2020-07-06/combined_hgp_dimensions_scored_output_202005
s3a://circleup-datascience/HGP/2020-06-03/combined_hgp_dimensions_scored_output_202004
s3a://circleup-datascience/HGP/2020-05-04/combined_hgp_dimensions_scored_output_202003
s3a://circleup-datascience/HGP/2020-04-09/combined_hgp_dimensions_scored_output_202002


In [14]:
outframe=pd.concat(filtered_df_dict.values(), ignore_index=True)

In [10]:
len(outframe)

11

In [11]:
outframe['fct_date'].value_counts()

2020-07    1
2020-02    1
2020-11    1
2020-10    1
2020-08    1
2021-01    1
2020-05    1
2020-09    1
2020-12    1
2020-04    1
2020-03    1
Name: fct_date, dtype: int64

In [15]:
outframe

,normalized_url,fct_date,parent_category_hgp_score
0,foodstirs.com,2021-01,0.001213
1,foodstirs.com,2020-12,0.000140
2,foodstirs.com,2020-11,0.000398
3,foodstirs.com,2020-10,0.002909
4,foodstirs.com,2020-09,0.001473
5,foodstirs.com,2020-08,0.002130
6,foodstirs.com,2020-07,0.000184
7,foodstirs.com,2020-05,0.007329
8,foodstirs.com,2020-04,0.253510
9,foodstirs.com,2020-03,0.025618


In [51]:
outframe.to_csv('cbd_historic_hgp_scores_210331.csv', index=False)